### University of Virginia
### DS 7200: Distributed Computing
### Lab: Supervised Learning
### Last Updated: August 20, 2023

---

#### Instructions

This project has two parts:
- Part I: Classification - build and apply a logistic regression model on the Wisconsin Breast Cancer dataset.
- Part II: Regression - build and apply a linear regression model on the California Housing dataset.

**Total Possible Points: 10**

---

#### Part I: Classification (5 POINTS)

Here are the specifications and grading breakdown:

- the target variable is `diagnosis`
- use `f1`, `f2` as predictors (1 PT)
- split data into 60% training set, 40% test set 
- standardize the predictors (1 PT)
- use seed=314 whenever a seed is needed
- fit a Logistic Regression model with an intercept (1 PT)
- compute and show the area under the ROC curve for the test set (2 PTS)

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import DoubleType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

DATA_FILEPATH = 'wisc_breast_cancer_w_fields.csv'

spark = SparkSession \
    .builder \
    .appName("Wisc BRCA") \
    .getOrCreate()

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/27 14:14:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Enter code and solution

In [2]:
# need to define a schema so that the data are not strings, I was getting an error with the vector assembler functions when strings
schema = StructType(
    [
        StructField('id', StringType(), False), 
        StructField('diagnosis', StringType(), False)
    ] + [StructField(f'f{x}', DoubleType(), False) for x in range(1, 31)]
)

In [3]:
# read in the file

df = spark.read.csv(DATA_FILEPATH, schema = schema, header = True)

df.show(2)

24/09/27 14:14:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+---------+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+
|    id|diagnosis|   f1|   f2|   f3|    f4|     f5|     f6|    f7|     f8|    f9|    f10|   f11|   f12|  f13|  f14|     f15|    f16|    f17|    f18|    f19|     f20|  f21|  f22|  f23|   f24|   f25|   f26|   f27|   f28|   f29|    f30|
+------+---------+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+
|842302|        M|17.99|10.38|122.8|1001.0| 0.1184| 0.2776|0.3001| 0.1471|0.2419|0.07871| 1.095|0.9053|8.589|153.4|0.006399|0.04904|0.05373|0.01587|0.03003|0.006193|25.38|17.33|184.6|2019.0|0.1622|0.6656|0.7119|0.2654|0.4601| 0.1189|
|842517|        M|20.57|17.77|132.9|1326.0|0.08474|0.07864|0.086

In [4]:
# show the schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- diagnosis: string (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- f3: double (nullable = true)
 |-- f4: double (nullable = true)
 |-- f5: double (nullable = true)
 |-- f6: double (nullable = true)
 |-- f7: double (nullable = true)
 |-- f8: double (nullable = true)
 |-- f9: double (nullable = true)
 |-- f10: double (nullable = true)
 |-- f11: double (nullable = true)
 |-- f12: double (nullable = true)
 |-- f13: double (nullable = true)
 |-- f14: double (nullable = true)
 |-- f15: double (nullable = true)
 |-- f16: double (nullable = true)
 |-- f17: double (nullable = true)
 |-- f18: double (nullable = true)
 |-- f19: double (nullable = true)
 |-- f20: double (nullable = true)
 |-- f21: double (nullable = true)
 |-- f22: double (nullable = true)
 |-- f23: double (nullable = true)
 |-- f24: double (nullable = true)
 |-- f25: double (nullable = true)
 |-- f26: double (nullable = true)
 |-- f27: double (n

In [5]:
# I need to look at the diagnosis column and turn to a 0 or 1 classification for logistic regression
# using string indexer to do this, I found this function here:
# https://datascience.stackexchange.com/questions/6268/how-to-convert-categorical-data-to-numerical-data-in-pyspark

indexer = StringIndexer(inputCol = 'diagnosis', outputCol = 'diagnosisIndex')
df = indexer.fit(df).transform(df)

In [6]:
# string indexer give 0 to "the most frequent label gets index 0"
# in this case B is 0 and M is 1
df.groupBy('diagnosis').count().show()

+---------+-----+
|diagnosis|count|
+---------+-----+
|        B|  357|
|        M|  212|
+---------+-----+



In [7]:
# confirm again by showing columns side by side
df.select(['diagnosis', 'diagnosisIndex']).show()

+---------+--------------+
|diagnosis|diagnosisIndex|
+---------+--------------+
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        M|           1.0|
|        B|           0.0|
+---------+--------------+
only showing top 20 rows



#### use f1, f2 as predictors (1 PT)

In [8]:
# select the target and predictor variables
assembler = VectorAssembler(
    inputCols = ['f1', 'f2'],
    outputCol = 'features'
)

# use the assembler
tr = assembler.transform(df.select(['diagnosisIndex', 'f1', 'f2']))
tr.select("*").show(2)

+--------------+-----+-----+-------------+
|diagnosisIndex|   f1|   f2|     features|
+--------------+-----+-----+-------------+
|           1.0|17.99|10.38|[17.99,10.38]|
|           1.0|20.57|17.77|[20.57,17.77]|
+--------------+-----+-----+-------------+
only showing top 2 rows



In [9]:
# split data into 60% training set, 40% test set
splits = tr.randomSplit([0.6, 0.4], seed = 314)
train = splits[0]
test = splits[1]

#### standardize the predictors (1 PT)

In [10]:
# standardize the predictor values
scaler = StandardScaler(
    inputCol="features", 
    outputCol="scaledFeatures", 
    withMean = True
)

# scale the training
train_scalerModel = scaler.fit(train)
train_scaled = train_scalerModel.transform(train)

# scale the testing
test_scalerModel = scaler.fit(test)
test_scaled = test_scalerModel.transform(test)

#### fit a Logistic Regression model with an intercept (1 PT)

In [11]:
# fit the logisitc regression
lr = LogisticRegression(
    labelCol='diagnosisIndex',
    featuresCol='scaledFeatures',
    maxIter=10, 
    regParam=0.3, 
    elasticNetParam=0.8
)

# Fit the model
lrModel = lr.fit(train_scaled)

In [12]:
# get the predictions from the the model on the test data
lrPred = lrModel.transform(test_scaled)
lrPred.select('probability', 'prediction').show(5, truncate = False)

+----------------------------------------+----------+
|probability                             |prediction|
+----------------------------------------+----------+
|[0.775085854099279,0.224914145900721]   |0.0       |
|[0.766078015770669,0.23392198422933097] |0.0       |
|[0.7584534742379524,0.2415465257620476] |0.0       |
|[0.758046455448967,0.24195354455103302] |0.0       |
|[0.7558404948265022,0.24415950517349783]|0.0       |
+----------------------------------------+----------+
only showing top 5 rows



In [13]:
lrPred.show(2)

+--------------+-----+-----+------------+--------------------+--------------------+--------------------+----------+
|diagnosisIndex|   f1|   f2|    features|      scaledFeatures|       rawPrediction|         probability|prediction|
+--------------+-----+-----+------------+--------------------+--------------------+--------------------+----------+
|           0.0| 7.76|24.54|[7.76,24.54]|[-1.8424000827161...|[1.23725504708864...|[0.77508585409927...|       0.0|
|           0.0|8.219| 20.7|[8.219,20.7]|[-1.7088204423750...|[1.18629635378102...|[0.76607801577066...|       0.0|
+--------------+-----+-----+------------+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



#### compute and show the area under the ROC curve for the test set (2 PTS)

In [14]:
# set up evaluator
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="rawPrediction",
    labelCol="diagnosisIndex",
    metricName="areaUnderPR"
)

aupr = evaluator.evaluate(lrPred)

# show the area under the curve
print('Area Under PR Curve:', aupr)

Area Under PR Curve: 0.922694056677676


In [15]:
# This is junk code on the start of trying to create the ROC curve but do not think we need it anymore

# # turn the data into an Rdd for the probabilities
# test_probs_and_labs = lrPred.select('probability', 'diagnosisIndex').rdd \
#                             .map(lambda x: (x.probability[1], x.diagnosisIndex))

# # get the curve
# test_probs_and_labs.show(2)

#### Part II: Regression (5 POINTS)

In this project, you will work with the California Home Price dataset to train a regression model and predict median home prices. Here are the specifications and grading breakdown:

- Scale the response variable median_house_value, dividing by 100000 (1 PT)

- Split data into train set (80%), test set (20%) using seed=314 (1 PT)

- Add new predictor: `rooms_per_household`

- In the training set, select all of these features and standardize them: (1 PT)

feats = ["total_bedrooms", 
         "population", 
         "households", 
         "median_income", 
         "rooms_per_household"]

- Fit a linear regression model on the training set with these parameters:

  - maxIter=10
  - regParam=0.3
  - elasticNetParam=0.8  


- Compute the MSE on the test set (2 PTS)

In [16]:
import os
import pandas as pd

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [17]:
DATA_FILEPATH2 = 'cal_housing_data_preproc_w_header.txt'

In [18]:
# generate the schema
schema = StructType(
    [
        StructField('median_house_value', DoubleType(), False), 
        StructField('median_income', DoubleType(), False),
        StructField('housing_median_age', DoubleType(), False),
        StructField('total_rooms', DoubleType(), False),
        StructField('total_bedrooms', DoubleType(), False),
        StructField('population', DoubleType(), False),
        StructField('households', DoubleType(), False),
        StructField('latitude', DoubleType(), False),
        StructField('longitude', DoubleType(), False)
    ]
)

# read in the data frame
house_data = spark.read.csv(DATA_FILEPATH2, schema = schema, header = True)

In [19]:
house_data.show(2)

+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+
|median_house_value|median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+
|          452600.0|       8.3252|              41.0|      880.0|         129.0|     322.0|     126.0|   37.88|  -122.23|
|          358500.0|       8.3014|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|   37.86|  -122.22|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+
only showing top 2 rows



In [20]:
house_data.printSchema()

root
 |-- median_house_value: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



#### Enter code and solution

#### Scale the response variable median_house_value, dividing by 100000 (1 PT)

In [21]:
# Scale the response variable median_house_value, dividing by 100000

house_data = house_data.withColumn('median_house_value', house_data.median_house_value / 100000)
house_data.show(2)

+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+
|median_house_value|median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+
|             4.526|       8.3252|              41.0|      880.0|         129.0|     322.0|     126.0|   37.88|  -122.23|
|             3.585|       8.3014|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|   37.86|  -122.22|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+
only showing top 2 rows



#### Split data into train set (80%), test set (20%) using seed=314 (1 PT)

In [22]:
# Split data into train set (80%), test set (20%) using seed=314

house_split = house_data.randomSplit([0.8, 0.2], seed = 314)
train = house_split[0]
test = house_split[1]

print('Train #', train.count())
print('Test #', test.count())

Train # 16472
Test # 4168


In [23]:
# Add new predictor: rooms_per_household
train = train.withColumn('rooms_per_household', train.total_rooms / train.households)
test = test.withColumn('rooms_per_household', test.total_rooms / test.households)

train.show(2)

+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+
|median_house_value|median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|rooms_per_household|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+
|           0.14999|        0.536|              36.0|       98.0|          28.0|      18.0|       8.0|   40.31|  -123.17|              12.25|
|           0.14999|       1.6607|              16.0|      255.0|          73.0|      85.0|      38.0|   39.71|  -122.74| 6.7105263157894735|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+
only showing top 2 rows



#### In the training set, select all of these features and standardize them: (1 PT)

In [24]:
# In the training set, select all of these features and standardize them
# feats = ["total_bedrooms", "population", "households", "median_income", "rooms_per_household"]

assembler = VectorAssembler(
    inputCols = ["total_bedrooms", "population", "households", "median_income", "rooms_per_household"],
    outputCol = 'features'
)

# use the assembler
train = assembler.transform(train)
test = assembler.transform(test)

# now create the scalar
scaler = StandardScaler(
    inputCol="features", 
    outputCol="scaledFeatures", 
    withMean = True
)

# scale the training
train_scalerModel = scaler.fit(train)
train_scaled = train_scalerModel.transform(train)

train_scaled.show(2)

+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+--------------------+--------------------+
|median_house_value|median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|rooms_per_household|            features|      scaledFeatures|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+--------------------+--------------------+
|           0.14999|        0.536|              36.0|       98.0|          28.0|      18.0|       8.0|   40.31|  -123.17|              12.25|[28.0,18.0,8.0,0....|[-1.2194405023784...|
|           0.14999|       1.6607|              16.0|      255.0|          73.0|      85.0|      38.0|   39.71|  -122.74| 6.7105263157894735|[73.0,85.0,38.0,1...|[-1.1120135412815...|
+------------------+-------------+------------------+-----------+--------------+

In [25]:
# Fit a linear regression model on the training set with these parameters:
# maxIter = 10
# regParam = 0.3
# elasticNetParam = 0.8

lr = LinearRegression(featuresCol='scaledFeatures',         # feature vector name
                      labelCol='median_house_value',  # target variable name
                      maxIter=10,
                      regParam=0.3, 
                      elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(train_scaled)

#### Compute the MSE on the test set (2 PTS)

In [26]:
## Compute the MSE on the test set
# have to scale the test

test_scaler = scaler.fit(test)
test_scaled = test_scaler.transform(test)

lrPred = lrModel.transform(test_scaled)
lrPred.show(2)

+------------------+------------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+--------------------+--------------------+------------------+
|median_house_value|     median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|rooms_per_household|            features|      scaledFeatures|        prediction|
+------------------+------------------+------------------+-----------+--------------+----------+----------+--------+---------+-------------------+--------------------+--------------------+------------------+
|           0.14999|            4.1932|              52.0|      803.0|         267.0|     628.0|     225.0|   34.24|  -117.86|  3.568888888888889|[267.0,628.0,225....|[-0.6209227643583...| 2.160114332761376|
|             0.225|0.7916999999999998|              52.0|      107.0|          79.0|     167.0|      53.0|   37.95|  -121.29|  2.018867924528302|[79.0,167.0,53.0,...|[

In [27]:
# actually getting the MSE
ev = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value")

# print the MSE
print('Mean Squared Error', ev.evaluate(lrPred, {ev.metricName: 'mse'}))

Mean Squared Error 0.7630394708043784
